In [2]:
import warnings
warnings.filterwarnings('ignore')
import nltk

In [3]:
import os
directory = "./abstracts/"
raw_docs = []
for filename in os.listdir(directory):
    try:
        f=open("./abstracts/"+ filename, "r")
        text = f.read()
        raw_docs.append(text)
    except Exception:
        print('didnt read paper' +filename)
    f.close()

didnt read paper.DS_Store
didnt read paper.ipynb_checkpoints


In [4]:
print('started')
from nltk.tokenize import word_tokenize
tokenized_docs = [word_tokenize(doc.lower()) for doc in raw_docs]
print('done')

started
done


In [5]:
#removing punctuation and numbers
import re
import string
regex = re.compile('[%s]' % re.escape(string.punctuation)) #see documentation here: http://docs.python.org/2/library/string.html

tokenized_docs_no_punctuation = []

for review in tokenized_docs:
    new_review = []
    for token in review:
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)
    
    tokenized_docs_no_punctuation.append(new_review)
print('done')

done


In [6]:
# removing common words; new_review has words with no punctuation
from nltk.corpus import stopwords

tokenized_docs_no_stopwords = []

for doc in tokenized_docs_no_punctuation:
    new_term_vector = []
    for word in doc:
        if not word in stopwords.words('english'):
            new_term_vector.append(word)
    
    tokenized_docs_no_stopwords.append(new_term_vector)

In [7]:
import nltk
withtag4elm = [nltk.pos_tag(paper) for paper in tokenized_docs_no_stopwords]

In [8]:
# Lemmatizing words with POS Tag

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

porter = PorterStemmer()
lematizer = WordNetLemmatizer()
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    elif treebank_tag.startswith('S'):
        return wordnet.ADJ_SAT
    else:
        return wordnet.NOUN

preprocessed_docs = []
for doc in withtag4elm:  
    wordstotheirROOT = []
    for word,tag in doc:
        wordstotheirROOT.append(lematizer.lemmatize(word, get_wordnet_pos(tag)))
    preprocessed_docs.append(wordstotheirROOT)

removing3z = []
for doc in preprocessed_docs: 
    no3 = []
    for word in doc:
        #print(word)
        if len(word) > 3 and word.isalpha():
            no3.append(word)
    removing3z.append(no3)

In [18]:
from gensim import corpora, models

list_of_list_of_tokens = removing3z
dictionary_LDA = corpora.Dictionary(list_of_list_of_tokens)
corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in list_of_list_of_tokens]

num_topics = 30
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=20, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

CPU times: user 1min 16s, sys: 487 ms, total: 1min 16s
Wall time: 1min 17s


In [19]:
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=10):
    print(str(i)+": "+ topic)
    print()

0: 0.024*"provide" + 0.021*"user" + 0.017*"support" + 0.016*"service" + 0.013*"architecture" + 0.013*"environment" + 0.012*"software" + 0.011*"spmv" + 0.011*"system" + 0.011*"compute"

1: 0.049*"trace" + 0.046*"page" + 0.032*"module" + 0.027*"memory" + 0.022*"time" + 0.019*"consistency" + 0.019*"pack" + 0.018*"warp" + 0.015*"protocol" + 0.013*"event"

2: 0.043*"application" + 0.041*"system" + 0.029*"performance" + 0.025*"resource" + 0.016*"compute" + 0.010*"workload" + 0.009*"high" + 0.009*"design" + 0.008*"storage" + 0.007*"service"

3: 0.044*"algorithm" + 0.035*"problem" + 0.021*"time" + 0.020*"bound" + 0.016*"number" + 0.014*"give" + 0.013*"approximation" + 0.010*"graph" + 0.010*"show" + 0.010*"model"

4: 0.059*"algorithm" + 0.041*"communication" + 0.030*"parallel" + 0.025*"implementation" + 0.017*"algorithms" + 0.016*"performance" + 0.014*"show" + 0.013*"present" + 0.012*"result" + 0.011*"paper"

5: 0.029*"node" + 0.022*"network" + 0.022*"problem" + 0.021*"model" + 0.018*"time" + 0

In [20]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=list_of_list_of_tokens, dictionary=dictionary_LDA, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4315402142652399


In [13]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary_LDA)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=list_of_list_of_tokens, dictionary=dictionary_LDA, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

With 3 topics each for a decade.

In [14]:
from gensim import corpora, models

list_of_list_of_tokens = removing3z
dictionary_LDA = corpora.Dictionary(list_of_list_of_tokens)
corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in list_of_list_of_tokens]

num_topics = 3 
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=20, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

CPU times: user 55.5 s, sys: 477 ms, total: 56 s
Wall time: 57.6 s


In [15]:
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=10):
    print(str(i)+": "+ topic)
    print()

0: 0.030*"algorithm" + 0.018*"problem" + 0.013*"time" + 0.012*"network" + 0.012*"graph" + 0.009*"parallel" + 0.009*"show" + 0.009*"number" + 0.009*"available" + 0.008*"model"

1: 0.021*"performance" + 0.020*"system" + 0.017*"application" + 0.007*"data" + 0.007*"compute" + 0.007*"simulation" + 0.007*"parallel" + 0.007*"model" + 0.006*"communication" + 0.006*"design"

2: 0.021*"memory" + 0.021*"data" + 0.017*"program" + 0.012*"performance" + 0.011*"parallel" + 0.010*"cache" + 0.008*"access" + 0.008*"code" + 0.007*"paper" + 0.007*"technique"



In [17]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=list_of_list_of_tokens, dictionary=dictionary_LDA, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.38577000969289005


In [16]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary_LDA)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

With 6 topics each for every 5 years.

In [24]:
from gensim import corpora, models

list_of_list_of_tokens = removing3z
dictionary_LDA = corpora.Dictionary(list_of_list_of_tokens)
corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in list_of_list_of_tokens]

num_topics = 10 
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=20, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=10):
    print(str(i)+": "+ topic)
    print()

CPU times: user 1min 2s, sys: 367 ms, total: 1min 2s
Wall time: 1min 3s
0: 0.018*"memory" + 0.017*"data" + 0.012*"computation" + 0.012*"gpus" + 0.011*"performance" + 0.009*"graph" + 0.009*"implementation" + 0.009*"application" + 0.008*"process" + 0.008*"propose"

1: 0.029*"memory" + 0.026*"cache" + 0.016*"performance" + 0.016*"processor" + 0.013*"schedule" + 0.011*"task" + 0.010*"execution" + 0.010*"time" + 0.009*"access" + 0.009*"propose"

2: 0.029*"program" + 0.026*"performance" + 0.019*"application" + 0.019*"parallel" + 0.017*"model" + 0.016*"code" + 0.009*"system" + 0.009*"present" + 0.008*"compiler" + 0.007*"tool"

3: 0.033*"system" + 0.023*"application" + 0.021*"data" + 0.018*"performance" + 0.010*"resource" + 0.010*"compute" + 0.008*"power" + 0.007*"design" + 0.007*"cluster" + 0.007*"storage"

4: 0.021*"algorithm" + 0.021*"parallel" + 0.016*"method" + 0.013*"problem" + 0.012*"matrix" + 0.010*"present" + 0.009*"data" + 0.008*"performance" + 0.008*"paper" + 0.008*"result"

5: 0.03

In [25]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=list_of_list_of_tokens, dictionary=dictionary_LDA, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4256868750655819


In [10]:
from gensim import corpora, models
from gensim.models import CoherenceModel

list_of_list_of_tokens = removing3z
dictionary_LDA = corpora.Dictionary(list_of_list_of_tokens)
corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in list_of_list_of_tokens]
coherences = []

for i in range(2,40):
    print(i)
    num_topics = i
    %time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                      id2word=dictionary_LDA, \
                                      passes=20, alpha=[0.01]*num_topics, \
                                      eta=[0.01]*len(dictionary_LDA.keys()))
    
    
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=list_of_list_of_tokens, dictionary=dictionary_LDA, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    coherences.append(coherence_lda)
    print('\nCoherence Score for ', i,' : ', coherence_lda)

2
CPU times: user 48.4 s, sys: 243 ms, total: 48.7 s
Wall time: 49.3 s

Coherence Score for  2  :  0.3261904424659178
3
CPU times: user 52.6 s, sys: 260 ms, total: 52.8 s
Wall time: 53.7 s

Coherence Score for  3  :  0.38339337499256126
4
CPU times: user 55.6 s, sys: 294 ms, total: 55.9 s
Wall time: 56.9 s

Coherence Score for  4  :  0.37955470727113566
5
CPU times: user 53.5 s, sys: 129 ms, total: 53.6 s
Wall time: 54 s

Coherence Score for  5  :  0.41273904208050444
6
CPU times: user 54.7 s, sys: 133 ms, total: 54.8 s
Wall time: 55 s

Coherence Score for  6  :  0.38454779616703516
7
CPU times: user 58.6 s, sys: 277 ms, total: 58.9 s
Wall time: 59.7 s

Coherence Score for  7  :  0.4389923582386092
8
CPU times: user 1min 7s, sys: 367 ms, total: 1min 8s
Wall time: 59.1 s

Coherence Score for  8  :  0.42612830618833897
9
CPU times: user 1min 13s, sys: 581 ms, total: 1min 14s
Wall time: 1min 8s

Coherence Score for  9  :  0.44131938441306773
10
CPU times: user 1min 13s, sys: 409 ms, total

In [ ]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary_LDA)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
from collections import Counter
list1=removing3z[0]
counts = Counter(list1)
print(counts)